In [1]:
import pandas as pd

In [2]:
import numpy as np
from lookup import lookup
import os
 
# Get the list of all files and directories
path = "C:/Users/dioge/Code/GitHub/2023-FRIB-TA-Summer-School/model-mixing/RBMSecar/Emulator/Results"
dir_list = os.listdir(path)

class CozyMatrix:
    def __init__(self,path):
        file = open(path,"r")
        self.matrices = []
        local_mat = 330*[0]
        count = 1
        for line in file:
            count += 1
            vec = line.split()
            if vec[0] == "L" or vec[0] == "P" or vec[0] == "A" or vec[0][0] == "-":
                continue
            if vec[0] == "I" and len(local_mat)!=0:
                self.matrices.append(local_mat)
                #print(local_mat)
                local_mat = 330*[0]
            elif vec[0] != "I":
                word = ""
                for i in range(3,len(vec)):
                    word+=vec[i]
                try:
                    local_mat[lookup[word]-1] = float(vec[1])
                except:
                    pass             

In [6]:
M1 = CozyMatrix(path+"/"+dir_list[2])

['6', '1.000000000000000', '1', '0', '0', '0', '0', '1', '0', '0', '0'] 00001000


In [5]:
lookup['0000100']
float(1.0)

1.0